### Step 1

Initialize the notebook, loading the configuration and importing libraries.

In [3]:
# Import the libraries required for the notebook
import asyncio

# Instatiate the config class
import sys
sys.path.append("../config")
from notebook_config import notebook_config
config = notebook_config()
# Load config from file
config.load_config_from_file()



### Step 2

Load the data from a FHIR example, in json format.

In [4]:
# Import a FHIR example
import wget
import os
import json
filename = 'example.json'

# Delete the file if it exists
if os.path.exists(filename):
    os.remove(filename)

# Download the file
wget.download('https://www.hl7.org/fhir/patient-example.json', filename)

# Load json file
try:
    with open(filename) as f:
        data = json.load(f)

    # Print the data
    print(data)

except FileNotFoundError:
    print(f"The file '{filename}' was not found.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


{'resourceType': 'Patient', 'id': 'example', 'text': {'status': 'generated', 'div': '<div xmlns="http://www.w3.org/1999/xhtml"><p style="border: 1px #661aff solid; background-color: #e6e6ff; padding: 10px;"><b>Jim </b> male, DoB: 1974-12-25 ( Medical record number: 12345\xa0(use:\xa0USUAL,\xa0period:\xa02001-05-06 --&gt; (ongoing)))</p><hr/><table class="grid"><tr><td style="background-color: #f3f5da" title="Record is active">Active:</td><td>true</td><td style="background-color: #f3f5da" title="Known status of Patient">Deceased:</td><td colspan="3">false</td></tr><tr><td style="background-color: #f3f5da" title="Alternate names (see the one above)">Alt Names:</td><td colspan="3"><ul><li>Peter James Chalmers (OFFICIAL)</li><li>Peter James Windsor (MAIDEN)</li></ul></td></tr><tr><td style="background-color: #f3f5da" title="Ways to contact the Patient">Contact Details:</td><td colspan="3"><ul><li>-unknown-(HOME)</li><li>ph: (03) 5555 6473(WORK)</li><li>ph: (03) 3410 5613(MOBILE)</li><li>ph

### Step 3

Initiate the Semantic Kernel and fire-up with OpenAI model.

In [5]:
# Load Semantic Kernel
import semantic_kernel
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

kernel = semantic_kernel.Kernel()
print("Kernel loaded.")

deployment = config.model
endpoint = config.endpoint
api_key = config.azure_api_key
kernel.add_chat_service("dv", AzureChatCompletion(deployment, endpoint, api_key))
print(f"Fire-up the kernel with {deployment}")

Kernel loaded.
Fire-up the kernel with gpt-35-turbo


### Step 4

Load plug-ins required to curate the congress notes.

In [6]:
plugins_directory = "../plugins-sk"

# Import the semantic functions
my_functions = kernel.import_semantic_skill_from_directory(plugins_directory, "FHIRanalysis")

print("Plugins loaded.")

Plugins loaded.


### Step 5

Execute the "FHIR summarize" process.

In [7]:
from semantic_kernel import ContextVariables

# Replace abbreviations and acronyms from the input text
input_str = str(data['text'])  # Ensure that the input is a string
fhir_summary = asyncio.create_task(kernel.run_async(my_functions["Summarize"], input_str=input_str))
fhir_summary_result = await fhir_summary
print(fhir_summary_result.result)

The provided resource is a Patient resource in FHIR standard. 

1. The resource is well defined as it contains all the necessary information about a patient, including their name, gender, date of birth, medical record number, active status, deceased status, alternate names, contact details, next-of-kin information, and links to managing organizations.

2. The resource is well designed as it follows the FHIR standard and includes all the relevant attributes and elements to represent a patient's information. The resource uses appropriate data types and references to other resources where necessary.

3. The resource implementation cannot be determined solely based on the provided text. To evaluate the resource's implementation, it would be necessary to assess how it is used in an actual system or application.
